In [1]:
import os
import sys
import numpy as np
sys.path.insert(0, os.path.realpath('../'))

In [2]:
import LDAQ

In [ ]:
# here we should show how to use the visualization object:
# - use of functions on channels of subplot
    # - use of built in functions
    # - use of custom functions
# - refresh rates, t_span, x_lim, y_lim
# - use case add_image, add_lines
# ...

In [3]:
# Create 1st serial acquisition source:
acq_arduino = LDAQ.serial_communication.SerialAcquisition(port="COM3", baudrate=250000, 
                                                acquisition_name="arduino",
                                                byte_sequence=(("int16", 2), ),
                                                start_bytes=  b"\xfa\xfb",
                                                end_bytes= b"\n",
                                                sample_rate=500,
                                                channel_names=["channel 1", "channel 2"])

# Here the arduino sends out bytes over serial where a line would look like this:
# b"\xfa\xfb\x00\x01\x00\x02\n

# Explanation:
# b"\xfa\xfb" are the start bytes
# b"\n" is the end byte
# b"\x00\x01" is the first sample (int16)
# b"\x00\x02" is the second sample (int16)